In [5]:
import scipy
import numpy as np
from scipy import integrate

class stencil:
    def __init__(self,theta,fun,lenX=5,lenY=5,N=1000,evalPoint="seeded"):
        self.alpha = np.zeros((lenX,lenY))
        h=1./N
        xP=fun.x(theta)
        yP=fun.y(theta)
        xMin=-np.float64((lenX-1))/2.
        xMax=-xMin+0.01
        yMin=-np.float64((lenY-1))/2.
        yMax=-yMin+0.01
        self.xL=np.arange(np.floor(xP)+xMin+0.5,np.floor(xP)+xMax+0.5,1.)
        self.yL=np.arange(np.floor(yP)+yMin+0.5,np.floor(yP)+yMax+0.5,1.)
        self.fun=fun
        iC=int((lenX-1)/2)
        jC=int((lenY-1)/2)
        if evalPoint=="nearest":
            xD=fun.nearestPoint(self.xL[iC],self.yL[jC],dx=1.,N=100)
            self.theta=fun.theta(xD[0],xD[1])
        elif evalPoint=="centroid":
            self.theta=fun.theta(self.xL[iC],self.yL[jC])
        elif evalPoint=="seeded":
            self.theta=theta
        self.K=fun.curvature(self.theta)
        self.n=fun.normal(self.theta)
         
        for i in range(lenX):
            for j in range(lenY):
                self.alpha[i,j]=fun.alphaCell(self.xL[i],self.yL[j],1.0,N)
    
    def convertZone(self):
        K=self.fun.curvature(self.theta)
        n=self.fun.normal(self.theta)
        alpha2=1*self.alpha
        iFac=1;jFac=1
        #xL=1*self.xL
        #yL=1*self.yL
        if n[0]<0:
            iFac=-1
            self.n[0]=fabs(n[0])
            #xL=flip(self.xL)
        if n[1]<0:
            jFac=-1
            self.n[1]=fabs(n[1])
            #yL=flip(self.yL)
        if K<0:
            alpha2=1-alpha2
            iFac*=-1;jFac*=-1
            self.K=fabs(K)
        iMax=int((self.alpha.shape[0]-1)/2)
        jMax=int((self.alpha.shape[1]-1)/2)
        
        alphaR=1*alpha2
        if fabs(n[0])<=fabs(n[1]):
            for i in range(-iMax,iMax+1):
                for j in range(-jMax,jMax+1):
                    alpha2[i+iMax,j+jMax]=alphaR[i*iFac+iMax,j*jFac+jMax]
        else:
            self.n[0]=fabs(n[1])
            self.n[1]=fabs(n[0])
            for i in range(-iMax,iMax+1):
                for j in range(-jMax,jMax+1):
                    alpha2[i+iMax,j+jMax]=alphaR[j*iFac+jMax,i*jFac+iMax]                  
        self.alpha=alpha2
        
        #return alpha2
        
from scipy.optimize import fsolve 
from scipy.optimize import bisect 
from scipy.optimize import brentq
from scipy.optimize import minimize
class shape:
    def x(self,theta):
        return 0.
    def x_t(self,theta):
        return 0.
    def x_tt(self,theta):
        return 0.
    def y(self,theta):
        return 0.
    def y_t(self,theta):
        return 0.
    def y_tt(self,theta):
        return 0.
    def theta(self,x,y):
        return 0.
    def normal(self,t):
        n_x=self.y_t(t)
        n_y=-self.x_t(t)
        mag=np.sqrt(n_x*n_x+n_y*n_y)
        return np.array([n_x,n_y])/mag
    def normalAngle(self,t): 
        n_x=self.y_t(t)
        n_y=-self.x_t(t)
        return arctan2(n_y,n_x)
    def curvature(self,t):
        x_t=self.x_t(t)
        x_tt=self.x_tt(t)
        y_t=self.y_t(t)
        y_tt=self.y_tt(t)
        return (x_t*y_tt-y_t*x_tt)/(x_t**2+y_t**2)**(1.5)
    def distance(self,x,y,dx):
        t=self.thetaMin(x,y,dx)
        return np.sqrt((x-self.x(t))**2.+(y-self.y(t))**2.)    
    def thetaMin(self,x,y,dx):
        #J=lambda t: np.sqrt((x-self.x(t))**2.+(y-self.y(t))**2.)
        #res=minimize(J, [self.theta(x,y)],method="CG")
        #return res.x
        J=lambda t: self.x_t(t)*(self.x(t)-x)+self.y_t(t)*(self.y(t)-y) 
        t0=self.theta(x,y)
        #dt=np.fabs(self.x(dx)-self.x(0.))
        #tL=np.linspace(t-dt,t+dt,50)
        #for t in tL:
        #    if J(t)>
        root=fsolve(J,t0)[0]
        if not np.isclose(J(root), 0.0):
            ValueError("root finding with fsolve failed.")
        return root
    def alphaCell(self,xC,yC,dx,N=100):
        #d=self.distance(xC,yC)
        #if d>1./np.sqrt(2)*dx:
        #    return self.value(xC,yC)
        h=dx/N
        xL=np.linspace(xC-dx/2.,xC+dx/2.,N+1)
        yL=np.linspace(yC-dx/2.,yC+dx/2.,N+1)
        sumA=np.zeros(4)
        yP=np.ones(N+1)*yL[0]
        sumA[0]=np.mean(self.value(xL,yP))
        yP=np.ones(N+1)*yL[-1]
        sumA[1]=np.mean(self.value(xL,yP))
        xP=np.ones(N+1)*xL[0]
        sumA[2]=np.mean(self.value(xP,yL))
        xP=np.ones(N+1)*xL[-1]
        sumA[3]=np.mean(self.value(xP,yL))
        sum=np.mean(sumA)
        if sum==0:
            return 0.
        elif sum==1:
            return 1.
        else:
            H=np.zeros(N+1)
            for n in range(N+1):
                xP=np.ones(N+1)*xL[n]
                H[n]=scipy.integrate.simps(self.value(xP,yL),yL)
                #H[n]=numpy.trapz(self.value(xP,yL),dx=h)
            return scipy.integrate.simps(H,xL)/dx/dx
        #return numpy.trapz(H,dx=h)
        
    def alphaCellMidPoint(self,xC,yC,dx,N=100):
        d=self.distance(xC,yC,dx)
        if d>1./np.sqrt(2)*dx:
            return self.value(xC,yC)
        Nf=np.float128(N)
        h=dx/Nf
        xL=np.linspace(xC-dx/2.,xC+dx/2.,N+1)
        yL=np.linspace(yC-dx/2.,yC+dx/2.,N+1)
        x2,y2=np.meshgrid(xL[:-1]+h/2.,yL[:-1]+h/2.)
        A=np.sum(self.value(x2.flatten(),y2.flatten()))
        return A/Nf/Nf
    
    
    def nearestPoint(self,xC,yC,dx,N=100):
        xP=np.zeros(2)
        xL=np.linspace(xC-dx/2.,xC+dx/2.,N+1)
        yL=np.linspace(yC-dx/2.,yC+dx/2.,N+1)
        d0=dx**2.
        for x in xL:
            for y in yL:
                if self.value(x,y)==1.:
                    d=(x-xC)**2.+(y-yC)**2.
                    if d<d0:
                        xP[0]=x
                        xP[1]=y
                        d=d0
        return xP

               
class circle(shape):
    def __init__(self, radius,origin=np.zeros(2)):
        self.R=radius
        self.Ox=origin[0]
        self.Oy=origin[1]
        #self.value=lambda x,y: (np.sign(self.R-np.sqrt(x*x+y*y))+1)/2.
        
    def theta(self,x,y):
        return np.arctan2(y-self.Oy,x-self.Ox)
        
    def value(self,x,y):
        xP=x-self.Ox
        yP=y-self.Oy
        return (np.sign(self.R**2.-(xP*xP+yP*yP))+1)/2.
    
    #def thetaMin(self,x,y):
    #    return self.theta(x,y)
        
        
    def x(self,theta):
        return self.R*np.cos(theta)+self.Ox
    def y(self,theta):
        return self.R*np.sin(theta)+self.Oy
    def normal(self,theta): 
        n_x=self.R*np.cos(theta) #self.y_t(theta)
        n_y=self.R*np.sin(theta) #-self.x_t(theta)
        mag=np.sqrt(n_x*n_x+n_y*n_y)
        return np.array([n_x,n_y])/mag
    def normalAngle(self,theta):        
        return theta #atan2(y-self.O[1],x-self.O[0])
    def curvature(self,theta):
        return 1/self.R
    

    
class ellipse(shape):
    def __init__(self,a,b,psi=0.):
        self.a=a
        self.b=b
        self.psi=psi
        #self.value=lambda x,y: (np.sign(self.R-np.sqrt(x*x+y*y))+1)/2.
        
    def theta(self,x,y):
        xR=x*np.cos(-self.psi)-y*np.sin(-self.psi)
        yR=x*np.sin(-self.psi)+y*np.cos(-self.psi)
        return np.arctan2(yR/self.b,xR/self.a)
     
    def value(self,x,y):
        theta=self.theta(x,y)
        R=x*x+y*y
        xE=self.x(theta)
        yE=self.y(theta)
        RE=xE*xE+yE*yE
        return (np.sign(RE-R)+1)/2.
        
    def x(self,theta):
        return self.a*np.cos(theta)*np.cos(self.psi)-self.b*np.sin(theta)*np.sin(self.psi)
    def x_t(self,theta):
        return -self.a*np.sin(theta)*np.cos(self.psi)-self.b*np.cos(theta)*np.sin(self.psi)
    def x_tt(self,theta):
        return -self.a*np.cos(theta)*np.cos(self.psi)+self.b*np.sin(theta)*np.sin(self.psi)
    def y(self,theta):
        return self.a*np.cos(theta)*np.sin(self.psi)+self.b*np.sin(theta)*np.cos(self.psi)
    def y_t(self,theta):
        return -self.a*np.sin(theta)*np.sin(self.psi)+self.b*np.cos(theta)*np.cos(self.psi)
    def y_tt(self,theta):
        return -self.a*np.cos(theta)*np.sin(self.psi)-self.b*np.sin(theta)*np.cos(self.psi)
    
class star(shape):
    def __init__(self,R,A,n,psi):
        self.R=R
        self.A=A
        self.n=n
        self.psi=psi
        
    def theta(self,x,y):
        xR=x*np.cos(-self.psi)-y*np.sin(-self.psi)
        yR=x*np.sin(-self.psi)+y*np.cos(-self.psi)
        #return np.arctan2(yR/self.b,xR/self.a)
        return np.arctan2(yR,xR)
     
    def value(self,x,y):
        theta=self.theta(x,y)
        R=x*x+y*y
        xE=self.x(theta)
        yE=self.y(theta)
        RE=xE*xE+yE*yE
        return (np.sign(RE-R)+1)/2.
        
    def x(self,theta):
        a=self.R+self.A*np.cos(self.n*theta)
        return a*np.cos(theta)*np.cos(self.psi)-a*np.sin(theta)*np.sin(self.psi)
    def x_t(self,theta):
        a=self.R+self.A*np.cos(self.n*theta)
        a_t=-self.A*self.n*np.sin(self.n*theta)
        return -a*np.sin(theta)*np.cos(self.psi)-a*np.cos(theta)*np.sin(self.psi)\
        +a_t*np.cos(theta)*np.cos(self.psi)-a_t*np.sin(theta)*np.sin(self.psi)
    def x_tt(self,theta):
        a=self.R+self.A*np.cos(self.n*theta)
        a_t=-self.A*self.n*np.sin(self.n*theta)
        a_tt=-self.A*self.n*self.n*np.cos(self.n*theta)
        x_tt=-a_t*np.sin(theta)*np.cos(self.psi)-a_t*np.cos(theta)*np.sin(self.psi)\
            +a_tt*np.cos(theta)*np.cos(self.psi)-a_tt*np.sin(theta)*np.sin(self.psi)
        x_tt+=-a*np.cos(theta)*np.cos(self.psi)+a*np.sin(theta)*np.sin(self.psi)\
            -a_t*np.sin(theta)*np.cos(self.psi)-a_t*np.cos(theta)*np.sin(self.psi)
        return x_tt
    def y(self,theta):
        a=self.R+self.A*np.cos(self.n*theta)
        return a*np.cos(theta)*np.sin(self.psi)+a*np.sin(theta)*np.cos(self.psi)
    def y_t(self,theta):
        a=self.R+self.A*np.cos(self.n*theta)
        a_t=-self.A*self.n*np.sin(self.n*theta)
        return -a*np.sin(theta)*np.sin(self.psi)+a*np.cos(theta)*np.cos(self.psi)\
        +a_t*np.cos(theta)*np.sin(self.psi)+a_t*np.sin(theta)*np.cos(self.psi)
    def y_tt(self,theta):
        a=self.R+self.A*np.cos(self.n*theta)
        a_t=-self.A*self.n*np.sin(self.n*theta)
        a_tt=-self.A*self.n*self.n*np.cos(self.n*theta)
        y_tt=-a_t*np.sin(theta)*np.sin(self.psi)+a_t*np.cos(theta)*np.cos(self.psi)\
            +a_tt*np.cos(theta)*np.sin(self.psi)+a_tt*np.sin(theta)*np.cos(self.psi)
        y_tt+=-a*np.cos(theta)*np.sin(self.psi)-a*np.sin(theta)*np.cos(self.psi)\
            -a_t*np.sin(theta)*np.sin(self.psi)+a_t*np.cos(theta)*np.cos(self.psi)
        return y_tt
    
    
class sinWave(shape):
    def __init__(self,A,k,psi=0.,phase=0.):
        self.k=k
        self.A=A
        self.psi=psi
        self.lamb=2*np.pi/k
        self.phase=phase
        #self.value=lambda x,y: (np.sign(self.R-np.sqrt(x*x+y*y))+1)/2.                                                                                                                                               

    def theta(self,x,y):
        #xR=x*np.cos(-self.psi)-y*np.sin(-self.psi)                                                                                                                                                                   
        xR=x*np.cos(-self.psi)-y*np.sin(-self.psi)
        return -xR

    def value(self,x,y):
        xR=x*np.cos(-self.psi)-y*np.sin(-self.psi)
        yR=x*np.sin(-self.psi)+y*np.cos(-self.psi)
        yW=self.A*np.cos(self.k*xR-self.phase)
        return (np.sign(yW-yR)+1)/2.

    def x(self,t):
        return -t*np.cos(self.psi)-self.A*np.cos(self.k*-t+self.phase)*np.sin(self.psi) #(theta-phi)/2*pi*self.lamb                                                                                                   
    def x_t(self,t):
        return -np.cos(self.psi)-self.A*self.k*np.sin(self.k*-t+self.phase)*np.sin(self.psi)
    def x_tt(self,t):
        return +self.A*self.k*self.k*np.cos(self.k*-t+self.phase)*np.sin(self.psi)
    def y(self,t):
        return -t*np.sin(self.psi)+self.A*np.cos(self.k*-t+self.phase)*np.cos(self.psi)
    def y_t(self,t):
        return -np.sin(self.psi)+self.A*self.k*np.sin(self.k*-t+self.phase)*np.cos(self.psi)
    def y_tt(self,t):
        return -self.A*self.k*self.k*np.cos(self.k*-t+self.phase)*np.cos(self.psi)
    
class spectralWave(shape):
    def __init__(self,A,k,phi,psi=0.):
        self.k=k
        self.A=A
        self.phase=phi
        self.psi=psi
        self.lamb=2*np.pi/k
        #self.value=lambda x,y: (np.sign(self.R-np.sqrt(x*x+y*y))+1)/2.
    
    def eta(self,x):
        eta=0.
        for i in range(self.A.shape[0]):
            eta+=self.A[i]*np.cos(self.k[i]*x+self.phase[i])
        return eta
        #theta=self.k*x+self.phase
        #eta=np.sum(self.A*np.cos(theta))
        #print(eta)
        #return eta
    def eta_x(self,x):
        eta_x=0.
        for i in range(self.A.shape[0]):
            eta_x+=self.A[i]*self.k[i]*np.sin(self.k[i]*x+self.phase[i])        
        return eta_x
        #return np.sum(-self.A*self.k*np.sin(self.k*x+self.phase))
    def eta_xx(self,x):
        eta_xx=0.
        for i in range(self.A.shape[0]):
            eta_xx-=self.A[i]*self.k[i]**2.*np.cos(self.k[i]*x+self.phase[i])
        return eta_xx
        #return np.sum(-self.A*self.k*self.k*np.cos(self.k*x+self.phase))
        
    def theta(self,x,y):
        xR=x*np.cos(-self.psi)-y*np.sin(-self.psi)
        return -xR
    
    def value(self,x,y):
        xR=x*np.cos(-self.psi)-y*np.sin(-self.psi)
        yR=x*np.sin(-self.psi)+y*np.cos(-self.psi)
        yW=self.eta(xR)
        return (np.sign(yW-yR)+1)/2. 

        
    def x(self,t):
        return -t*np.cos(self.psi)-self.eta(-t)*np.sin(self.psi) #(theta-phi)/2*pi*self.lamb
    def x_t(self,t):
        return -np.cos(self.psi)-self.eta_x(-t)*np.sin(self.psi)
    def x_tt(self,t):
        return -self.eta_xx(-t)*np.sin(self.psi)
    def y(self,t):
        return -t*np.sin(self.psi)+self.eta(-t)*np.cos(self.psi)
    def y_t(self,t):
        return -np.sin(self.psi)+self.eta_x(-t)*np.cos(self.psi)
    def y_tt(self,t):
        return self.eta_xx(-t)*np.cos(self.psi)
    
class JONSWAPWave(spectralWave):
    def __init__(self,Hs,lambda_p,gamma,Nkx=100,psi=0.,seed=10):
        self.psi=psi
        beta=0
        g=9.81
        alpha=1.
        omega_p=np.sqrt(g*2*pi/lambda_p)
        self.omega_p=omega_p
        self.Hs=Hs
        self.gamma=gamma
        #lambda_p=2*pi/(omega_p**2./g)
        #omega_p=2*pi/Tp
        #print("lambda_p=",lambda_p)
        
        random.seed(seed)

        self.alpha=1
        omegaL=np.arange(0.01,50.01,0.01)*omega_p
        dOmega=omegaL[1]-omegaL[0]
        G=0
        for i in range(omegaL.shape[0]):
            G+=self.Gw(omegaL[i])
        Hs2=4*np.sqrt(G*dOmega)
        self.alpha=(Hs/Hs2)**2.
        print("self.alpha",self.alpha)

        Lx=20*lambda_p
        self.Lx=Lx
        dkx=2*pi/Lx
        self.A=np.zeros(Nkx+1)
        self.k=np.zeros(Nkx+1)
        self.phase=np.zeros(Nkx+1)
        for i in range(1,Nkx+1): 
            self.k[i]=i*dkx
            omega=np.sqrt(g*self.k[i])
            Gw1=self.Gw(omega)
            self.A[i]=sqrt(2*sqrt(g/self.k[i])*Gw1*dkx)
            #self.A[i]=np.sqrt(2*(g**2/(2*omega**3.))*Gw1*dkx)
            self.phase[i]=np.random.rand(1)*2*np.pi
        #print(Gw1,self.A[i])
        print("E(k)=%f"%(sum(self.A**2)/2.))
        [omega,G]=self.spectrum()
        dw=omega[1]-omega[0]
        print("E(w)=%f"%(sum(G*dw)))
        
    def getLx(self):
        return self.Lx

    
    def Gw(self,omega):
        #omega_p=2*pi/Tp
        sigma=0.07
        if omega>self.omega_p:
            sigma=0.09
        if self.gamma==1.:
            h=1.
        else:
            h=self.gamma**np.exp(-1/2*((omega-self.omega_p)/(sigma*self.omega_p))**2.)       
        return self.alpha*self.Hs**2*self.omega_p**4.*omega**(-5)*np.exp(-5/4.*(omega/self.omega_p)**(-5))*h
    def spectrum(self):
        omegaL=np.arange(0.01,50.01,0.01)*self.omega_p
        GL=np.zeros(omegaL.shape[0])
        for i in range(omegaL.shape[0]):
            GL[i]=self.Gw(omegaL[i])
        return omegaL,GL
    
    
class PowerWave(spectralWave):
    def __init__(self,Hs,lambda_p,Nkp=10,psi=0.):
        self.psi=psi
        self.k_p=2*np.pi/lambda_p
        self.Hs=Hs
        dk=0.1*self.k_p
        self.k=np.arange(dk,self.k_p*Nkp+0.00001,dk)
        Nk=self.k.shape[0]
        E=np.exp(-1.25*(self.k_p/self.k)**4.)*self.k**-5.
        Hs2=4*np.sqrt(np.sum(E*dk))
        self.alpha=Hs/Hs2
        self.A=self.alpha*np.sqrt(2*E*dk)
        self.phase=np.random.uniform(0,2*pi,Nk)
        self.E=E
    
    



In [ ]:
import tensorflow as tf
from tensorflow import keras                                                                                                                                                                                                                                                                       
from tensorflow.keras.models import load_model

class mlpCurvature:
    def __init__(self,modelPath,zoneModel=True,invertCurvature=True,NS=5,symmetric=False,positiveNormals=True,averageZones=True):
        self.model=load_model(modelPath)
        self.zoneModel=zoneModel
        self.invertCurvature=invertCurvature
        self.positiveNormals=positiveNormals
        self.NS=NS
        self.symmetric=symmetric
        self.averageZones=averageZones
        if self.symmetric:
            self.invertCurvature=False
        if modelPath.find("3x3")>-1:
            self.iMax=1;self.jMax=1
        elif modelPath.find("3x5")>-1:
            self.iMax=1;self.jMax=2
        elif modelPath.find("5x3")>-1:
            self.iMax=2;self.jMax=1
        elif modelPath.find("5x5")>-1:
            self.iMax=2;self.jMax=2
        else:
            ValueError("Stencil of the model %s is not recognized."%modelPath)
            
        iC=0
        self.indices=np.zeros((2*self.iMax+1)*(2*self.jMax+1),dtype=int)
        for i in range(-self.iMax,self.iMax+1,1):
            for j in range(-self.jMax,self.jMax+1,1):
                self.indices[iC]=int((i+2)+self.NS*(j+2))
                iC+=1
        
    def normals(self,alpha):
        if 0:
            Hx=np.zeros(3)
            Hy=np.zeros(3)
            m=int((alpha.shape[0]-1)/2)
            n=int((alpha.shape[1]-1)/2)
            for i in range(m-1,m+2):
                for j in range(n-1,n+2):
                    Hy[i-m+1]+=alpha[i,j]
            for j in range(n-1,n+2):
                for i in range(m-1,m+2):
                    Hx[j-n+1]+=alpha[i,j]

            n=np.zeros(2)
            n[0]=-(Hy[2]-Hy[0])
            n[1]=-(Hx[2]-Hx[0])
            
        young=reconScheme("young")
        dx=1.
        return young.predict(alpha,dx)
        
    def curvatureSign(self,alpha): #,n):
        H=np.zeros(3)
        n=self.normals(alpha)
        #normalDir=getNormalDir(self,alpha)
        
        #if normalDir==1: 
        if np.fabs(n[0])<=np.fabs(n[1]):
            m=int((alpha.shape[0]-1)/2)
            iC=0
            for i in range(m-1,m+2):
                for j in range(m-2,m+3):
                    H[iC]+=alpha[i,j]
                iC+=1
        else:
            m=int((alpha.shape[1]-1)/2)
            iC=0
            for j in range(m-1,m+2):
                for i in range(m-2,m+3):
                    H[iC]+=alpha[i,j]
                iC+=1
        return -np.sign((H[2]-2*H[1]+H[0]))
        #return np.sign((H[2]-2*H[1]+H[0]))
        
    def convertZone(self,alpha): 
        n=self.normals(alpha)
        alpha2=1*alpha
        iFac=1;jFac=1
        if self.positiveNormals:
            if n[0]<0:
                iFac=-1
                #print("found negative x-normal...")
            if n[1]<0:
                jFac=-1
                #print("found negative y-normal...")
        if self.invertCurvature:
            K=self.curvatureSign(alpha)
            if K<0:
                alpha=1-alpha
                iFac*=-1;jFac*=-1
            #print("found negative curvature...")

        iMax=int((alpha.shape[0]-1)/2)
        jMax=int((alpha.shape[1]-1)/2)
        if np.fabs(n[0])<=np.fabs(n[1]):
            for i in range(-iMax,iMax+1):
                for j in range(-jMax,jMax+1):
                    alpha2[i+iMax,j+jMax]=alpha[i*iFac+iMax,j*jFac+jMax]
        else:
            #print("x-normal dominant, rotating...")
            for i in range(-iMax,iMax+1):
                for j in range(-jMax,jMax+1):
                    alpha2[i+iMax,j+jMax]=alpha[j*iFac+jMax,i*jFac+iMax]
                    #alpha2[i+iMax,j+jMax]=alpha[j*jFac+jMax,i*iFac+iMax]  
        return alpha2
        
    
    def predict(self,alpha):
        if self.zoneModel:
            if self.symmetric:
                alpha=self.convertZone(alpha)
                alpha=2.*(alpha-0.5)
                iMax=int((alpha.shape[0]-1)/2)
                jMax=int((alpha.shape[1]-1)/2)
                NInput=alpha.shape[0]*alpha.shape[1]
                alpha1D=np.zeros(NInput)
                iC=0
                for i in range(-self.iMax,self.iMax+1,1):
                    for j in range(-self.jMax,self.jMax+1,1):                                                                                                                                                                                   
                        alpha1D[iC]=0.25*(alpha[i+iMax,j+jMax]+alpha[-i+iMax,j+jMax]+alpha[i+iMax,-j+jMax]+alpha[-i+iMax,-j+jMax])
                        iC+=1
                alpha1D=alpha1D[:iC]
                curvature=self.model.predict(np.array([alpha1D,]))
            else:
                if not self.positiveNormals:
                    alpha=self.convertZone(alpha)
                    alpha=2.*(alpha-0.5)
                    iMax=int((alpha.shape[0]-1)/2)
                    jMax=int((alpha.shape[1]-1)/2)
                    NInput=(2*self.iMax+1)*(2*self.jMax+1) #alpha.shape[0]*alpha.shape[1]
                    
                    if self.averageZones:
                        alpha1D=np.zeros((NInput,4))
                        iC=0 
                        for i in range(-self.iMax,self.iMax+1,1):
                            for j in range(-self.jMax,self.jMax+1,1):                                                                                                                                                                           
                                alpha1D[iC,0]=alpha[i+iMax,j+jMax]
                                alpha1D[iC,1]=np.fliplr(alpha)[i+iMax,j+jMax] #alpha[-i+iMax,j+jMax]
                                alpha1D[iC,2]=np.flipud(alpha)[i+iMax,j+jMax] #alpha[i+iMax,-j+jMax]
                                alpha1D[iC,3]=np.flipud(np.fliplr(alpha))[i+iMax,j+jMax] #alpha[-i+iMax,-j+jMax]
                                iC+=1

                        curvature=0.25*self.model.predict(np.array([alpha1D[:,0],]))
                        curvature+=0.25*self.model.predict(np.array([alpha1D[:,1],]))
                        curvature+=0.25*self.model.predict(np.array([alpha1D[:,2],]))
                        curvature+=0.25*self.model.predict(np.array([alpha1D[:,3],]))
                    else:
                        alpha1D=np.zeros(NInput)
                        iC=0 
                        for i in range(-self.iMax,self.iMax+1,1):
                            for j in range(-self.jMax,self.jMax+1,1):                                                                                                                                                                           
                                alpha1D[iC]=alpha[i+iMax,j+jMax]
                                iC+=1
                        curvature=self.model.predict(np.array([alpha1D,]))
                else:
                    alpha1D=self.convertZone(alpha).flatten(order='F') #.reshape(1,alpha.shape[0]*alpha.shape[1])
                    if self.invertCurvature:
                        signFac=self.curvatureSign(alpha)
                    else:
                        signFac=1.
                    curvature=signFac*self.model.predict(np.array([alpha1D[self.indices],]))
        else:
            if self.symmetric:
                #print("raw sym model")
                alpha=2.*(alpha-0.5)
                iMax=int((alpha.shape[0]-1)/2)
                jMax=int((alpha.shape[1]-1)/2)
                NInput=alpha.shape[0]*alpha.shape[1]
                alpha1D=np.zeros(NInput)
                iC=0
                for i in range(-self.iMax,self.iMax+1,1):
                    for j in range(-self.jMax,self.jMax+1,1):                                                                                                                                                                                   
                        alpha1D[iC]=0.125*(alpha[i+iMax,j+jMax]+alpha[-i+iMax,j+jMax]+alpha[i+iMax,-j+jMax]+alpha[-i+iMax,-j+jMax])
                        alpha1D[iC]+=0.125*(alpha[j+iMax,i+jMax]+alpha[-j+iMax,i+jMax]+alpha[j+iMax,-i+jMax]+alpha[-j+iMax,-i+jMax])
                        iC+=1
                alpha1D=alpha1D[:iC]
                curvature=self.model.predict(np.array([alpha1D,]))
            else:
                alpha1D=alpha.flatten(order='F') #.reshape(1,alpha.shape[0]*alpha.shape[1])
                curvature=self.model.predict(np.array([alpha1D[self.indices],]))
        
        return curvature
    
class mlpNormal(mlpCurvature):
    def predict(self,alpha):
        if self.zoneModel:
            alpha1D=self.convertZone(alpha).flatten(order='F').reshape(1,alpha.shape[0]*alpha.shape[1])
        else:
            alpha1D=alpha.flatten(order='F').reshape(1,alpha.shape[0]*alpha.shape[1])

        iC=0
        indices=np.zeros((2*self.iMax+1)*(2*self.jMax+1),dtype=int)
        for i in range(-self.iMax,self.iMax+1,1):
            for k in range(-self.jMax,self.jMax+1,1):
                indices[iC]=int((i+2)+self.NS*(k+2))
                iC+=1
        n=np.zeros(2)
        nx=self.model.predict(alpha1D[:,indices])
        nMag=np.sqrt(nx**2+1)
        n[0]=nx/nMag
        n[1]=1./nMag
        nE=self.normals(alpha)
        K=self.curvatureSign(alpha)
        if self.zoneModel:
            if np.fabs(nE[0])<=np.fabs(nE[1]):
                n[0]*=np.sign(nE[0])
                n[1]*=np.sign(nE[1])
            else:
                nR=1.*n
                n[0]=np.sign(nE[0])*nR[1]
                n[1]=np.sign(nE[1])*nR[0]
            #if K<0:
            #    n[0]*=-1.
            #    n[1]*=-1.
            
        return n
    
class heightFunction:
    def predict(self,alpha,dx,nDir=1):
        Nx=alpha.shape[0]
        Ny=alpha.shape[1]
        if Nx>Ny: 
            nDir=0
        H=np.zeros(3)
        if nDir==1:   
            for i in range(Nx):
                for j in range(Ny):
                    H[i]+=alpha[i,j]*dx
        else:
            for j in range(Ny):
                for i in range(Nx):
                    H[j]+=alpha[i,j]*dx
                    
        Hx=(H[2]-H[0])/(2*dx)
        Hxx=(H[2]-2.*H[1]+H[0])/dx**2
                    
        return -Hxx/(1.+Hx**2)**1.5
    
class reconScheme:
    def __init__(self,scheme):
        self.scheme=scheme
    def predict(self,alpha,dx):
        n=np.zeros(2)
        m=int((alpha.shape[0]-1)/2)
        k=int((alpha.shape[1]-1)/2)
        if self.scheme=='ccd':
            n=self.ccd(alpha,dx,m,k)
        elif self.scheme=='young':
            n=self.young(alpha,dx,m,k)  
        elif self.scheme=='myc':
            nC=self.ccd(alpha,dx,m,k)
            nY=self.young(alpha,dx,m,k)          
            if np.fabs(nC[0])<=np.fabs(nC[1]):
                if np.fabs(nC[1])>np.fabs(nY[1]):
                    n=nY
                else:
                    n=nC
            else:
                if np.fabs(nC[0])>np.fabs(nY[0]):
                    n=nY
                else:
                    n=nC
        return n
    
    def young(self,alpha,dx,m,k):
        A=alpha
        n=np.zeros(2)
        n[0]=1./8./dx*(A[m+1,k+1]-A[m-1,k+1]+2.0*A[m+1,k+0]-2.0*A[m-1,k+0]+A[m+1,k-1]-A[m-1,k-1]);
        n[1]=1./8./dx*(A[m+1,k+1]-A[m+1,k-1]+2.0*A[m+0,k+1]-2.0*A[m+0,k-1]+A[m-1,k+1]-A[m-1,k-1]);
        n/=-np.sqrt(n[0]**2+n[1]**2.) 
        return n
    def ccd(self,alpha,dx,m,k):
        Hy=np.zeros(3)
        for i in range(m-1,m+2):
            for j in range(k-1,k+2):
                Hy[i-m+1]+=alpha[i,j]
        Hx=np.zeros(3)
        for j in range(k-1,k+2):
            for i in range(m-1,m+2):
                Hx[j-k+1]+=alpha[i,j]

        Hy_x=(Hy[2]-Hy[0])/2.
        Hx_y=(Hx[2]-Hx[0])/2.
        n=np.zeros(2)
        if np.fabs(Hy_x)<=np.fabs(Hx_y):
            n[0]=-Hy_x
            nMag=np.sqrt(n[0]**2.+1)
            n[1]=np.sign(-Hx_y)
            #n*=np.sign(-Hx_y)
        else:
            n[1]=-Hx_y
            nMag=np.sqrt(n[1]**2.+1)
            n[0]=np.sign(-Hy_x)
        n/=nMag  
        return n
    
    
class mlpCurvatureOld(mlpCurvature):
    #def __init__(self,modelPath,zoneModel=True):
    #    self.modelCurv=load_model(modelPath)
    #    self.zoneModel=zoneModel
    def __init__(self,modelPath,zoneModel=True):
        self.model=load_model(modelPath)
        self.zoneModel=zoneModel
        self.NS=5
        self.iMax=2;
        self.jMax=1;
        
        iC=0
        self.indices=np.zeros((2*self.iMax+1)*(2*self.jMax+1),dtype=int)
        for i in range(-self.iMax,self.iMax+1,1):
            for j in range(-self.jMax,self.jMax+1,1):
                self.indices[iC]=int((i+2)+self.NS*(j+2))
                iC+=1
        
        
    def convertZone(self,alpha):
        K=self.curvatureSign(alpha)
        n=self.normals(alpha)
        alpha2=1*alpha
        iFac=1;jFac=1
        if n[0]>0:
            iFac=-1
        if n[1]<0:
            jFac=-1
        if K<0:
            alpha=1-alpha
            iFac*=-1;jFac*=-1

        iMax=int((alpha.shape[0]-1)/2)
        jMax=int((alpha.shape[1]-1)/2)
        if fabs(n[0])<=fabs(n[1]):
            for i in range(-iMax,iMax+1):
                for j in range(-jMax,jMax+1):
                    alpha2[i+iMax,j+jMax]=alpha[i*iFac+iMax,j*jFac+jMax]
        else:
            iFac*=-1;jFac*=-1
            for i in range(-iMax,iMax+1):
                for j in range(-jMax,jMax+1):
                    alpha2[i+iMax,j+jMax]=alpha[j*iFac+jMax,i*jFac+iMax]                  
        return 2*(alpha2-0.5)
  

In [11]:
import os
modelPath="trainElementaryShapes/models_NInt200"
models=os.listdir(modelPath)
models = list(filter(lambda a: 'circ' in a, models))
#print(models)

In [12]:
def testNormModels(alphaPath,NInt,modelPath="trainElementaryShapes/models_NInt",eps=10e-6,modelType="_"):
    modelPath="%s%d"%(modelPath,NInt)
    alphaFile="trainElementaryShapes/testDatasets/%s.pkl"%alphaPath
    fName = open(alphaFile,'rb')
    data=pickle.load(fName)
    fName.close()
    NL=data["NL"]
    print(NL)

    wDir="trainElementaryShapes/testResults_NInt%d"%(NInt)
    if not os.path.exists(wDir):
        os.mkdir(wDir)
    #fName="%s/%s_L1_Der1.dat"%(wDir,alphaPath)
    fName="%s/%s_eps%g_L1_Der1.dat"%(wDir,alphaPath,eps)
    #fName="trainElementaryShapes/testResults/%s_L1_Der1.dat"%alphaPath
    if os.path.exists(fName):
        df=pd.read_csv(fName)
    else:
        df=pd.DataFrame({'name': ['model']})
        for n in range(len(NL)):
            df.insert(n+1,"%d"%NL[n],[0.])

    models=os.listdir(modelPath)
    models = list(filter(lambda a: modelType in a, models))
    models.insert(0,"reconScheme")
    scheme='myc'
    for model in models:
        if df[df.iloc[:,0]==model].shape[0]<1:
            if model=="reconScheme":
                recon=reconScheme(scheme)
            else:
                try:
                    if model.find("Raw")>-1:
                        mlpN=mlpNormal("%s/%s/model_Der1.h5"%(modelPath,model),zoneModel=False)
                    else:
                        mlpN=mlpNormal("%s/%s/model_Der1.h5"%(modelPath,model))
                except:
                    continue
            print("new model: %s"%model)
            df2=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df2.insert(n+1,"%d"%NL[n],[0.])
            L1=np.zeros(len(NL))
            iC=0
            for N in data["NL"]:
                x2D=data["x2D_%d"%N]
                y2D=data["y2D_%d"%N]
                alpha=data["alpha_%d"%N]
                kappa=data["kappa_%d"%N]
                nx=data["nx_%d"%N]
                ny=data["ny_%d"%N]
                nMag=(nx**2+ny**2)
                nx/=nMag
                ny/=nMag
                dx=x2D[1,0]-x2D[0,0]

                nxML=np.zeros(x2D.shape)*np.nan
                nyML=np.zeros(x2D.shape)*np.nan

                if model=="reconScheme":
                    for i in range(1,x2D.shape[0]-1):
                        for j in range(1,y2D.shape[1]-1):
                            if eps<alpha[i,j]<1-eps:
                                nML=recon.predict(alpha[i-1:i+2,j-1:j+2],dx)
                                nxML[i,j]=nML[0]
                                nyML[i,j]=nML[1]
                    df2['name']=scheme
                else:
                    NInt=0.
                    for i in range(2,x2D.shape[0]-2):
                        for j in range(2,y2D.shape[1]-2):
                            if eps<alpha[i,j]<1-eps:
                                nML=mlpN.predict(alpha[i-2:i+3,j-2:j+3])
                                nxML[i,j]=nML[0]
                                nyML[i,j]=nML[1]
                    df2['name']=model

                err=1.-(nxML*nx+nyML*ny)
                L1[iC]=np.nanmean(np.fabs(err))
                iC+=1

            df2.iloc[0,1:]=L1
            if df.iloc[0,1]>0.0:
                df=df.append(df2,ignore_index=True)
            else:
                df=df2
            df=df.sort_values(by=['%s'%NL[-1]])
            df.to_csv(path_or_buf=fName, index=False)
    return df

In [53]:
import pickle
import pandas as pd
from pandas import read_csv
import os
import glob


def generateTestDatabase(dbPath="trainElementaryShapes/testDatasets",eps=1e-8):
    files=os.listdir(dbPath)
    files = list(filter(lambda a: "pkl" in a, files))
        
    db={}
    db["case"]=[]
    db["alpha"]=[]
    db["kappa"]=[]
    db["nx"]=[]
    db["ny"]=[]
    db["dx"]=[]
    for file in files:
        if file.find("allCases")>-1:
            continue
        print("Case: ",file)
        alphaFile="%s/%s"%(dbPath,file)
        fName = open(alphaFile,'rb')
        data=pickle.load(fName)
        fName.close()

        for N in data["NL"]:
            x2D=data["x2D_%d"%N]
            y2D=data["y2D_%d"%N]
            alpha=data["alpha_%d"%N]
            kappa=data["kappa_%d"%N]
            nx=data["nx_%d"%N]
            ny=data["ny_%d"%N]
            dx=x2D[1,0]-x2D[0,0]
                
            Nx=x2D.shape[0]

            for i in range(3,x2D.shape[0]-3):
                for j in range(3,y2D.shape[1]-3):
                    if eps<alpha[i,j]<1-eps and not np.isnan(kappa[i,j]):
                        db["case"].append(file)
                        db["alpha"].append(alpha[i-3:i+4,j-3:j+4])
                        db["kappa"].append(kappa[i,j]*dx)
                        db["nx"].append(nx[i,j])
                        db["ny"].append(ny[i,j])
                        db["dx"].append(dx)
                        
    print("Sample size = ",len(db["kappa"]))
    fName="%s/allCases_%s.pkl"%(dbPath,eps)
    with open(fName, "wb") as fp:
        pickle.dump(db,fp)
                            
def evaluateCurvDatabase(model,dbPath="trainElementaryShapes/testDatasets/allCases_1e-08.pkl",modelPath="trainElementaryShapes/models_NInt200"):
    fName = open(dbPath,'rb')
    db=pickle.load(fName)
    fName.close()
    
    NL=len(db["alpha"])
    
    if model=="heightFunction33" or model=="heightFunction35" or model=="heightFunction37" or model=="heightFunction":
        hf=heightFunction()
    elif model[:3]=="sgd":
        mlpC=mlpCurvatureOld("%s/%s/model_Curv.h5"%(modelPath,model))
    else:
        if model.find("Raw")>-1:
            mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model),zoneModel=False)
        else:
            mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model))
            
    results={}
    results["kappa"]=db["kappa"]
    results["kappaML"]=[]
    results["mse"]=0.
    results["mae"]=0.
    for i in range(NL):
        if 0: #kmod(i,5000)==0:
            print(i,db["case"][i])
            print(results["mae"],results["mse"])
        dx=db["dx"][i]
        alpha=db["alpha"][i]
        young=reconScheme("young")
        n=young.predict(alpha[2:5,2:5],dx)
        nx=n[0];ny=n[1]
        nDir=1
        if np.fabs(nx)>np.fabs(ny):
            nDir=0
        if model=="heightFunction33":
             results["kappaML"].append(hf.predict(alpha[2:5,2:5],dx,nDir=nDir)*dx)
        elif model=="heightFunction35":
            if nDir==1:
                results["kappaML"].append(hf.predict(alpha[2:5,1:6],dx)*dx)
            else:
                results["kappaML"].append(hf.predict(alpha[1:6,2:5],dx)*dx)
        elif model=="heightFunction":
            if nDir==1:
                results["kappaML"].append(hf.predict(alpha[2:5,:],dx)*dx)
            else:
                results["kappaML"].append(hf.predict(alpha[:,2:5],dx)*dx)                 
        else:
            k=mlpC.predict(alpha[1:6,1:6])[0][0]
            #print(k)
            results["kappaML"].append(k)
                
        results["mse"]+=(results["kappaML"][i]-results["kappa"][i])**2.
        results["mae"]+=np.fabs(results["kappaML"][i]-results["kappa"][i])
        if np.isnan(results["mse"]):
            print("NAN detected.",db["case"][i],db["dx"][i],db["kappa"][i],db["alpha"][i])                   

    results["mse"]/=float(NL)
    results["mae"]/=float(NL)
    return results


In [58]:
def evalCurvModels(modelPath="trainElementaryShapes/models_NInt200",dbPath="trainElementaryShapes/testDatasets"
                   ,dbFile="allCases_1e-08.pkl",modelType="_",fName="_"):

    wDir="trainElementaryShapes/testResults_NInt200"
    if not os.path.exists(wDir):
        os.mkdir(wDir)
    fName="%s/%s_%s_Curv.dat"%(wDir,dbFile,modelType)

    models=os.listdir(modelPath)
    models = list(filter(lambda a: modelType in a, models))
    models.insert(0,"heightFunction33")
    models.insert(1,"heightFunction35")
    models.insert(2,"heightFunction")
    for model in models:
        if os.path.exists(fName):
            df=pd.read_csv(fName)
        else:
            df=pd.DataFrame({'name': ['model']})
            df.insert(1,"mse",[0.])
            df.insert(2,"mae",[0.])
                
        if df[df.iloc[:,0]==model].shape[0]<1:
            df2=pd.DataFrame({'name': ['model']})
            df2.insert(1,"mse",[0.])
            df2.insert(2,"mae",[0.])
            
            try:
                print(model)
                res=evaluateCurvDatabase(model,dbPath="%s/%s"%(dbPath,dbFile))
            except:
                continue

            df2['name']=model
            df2.iloc[0,1]=res["mse"]
            df2.iloc[0,2]=res["mae"]
            if df.iloc[0,1]>0.0:
                df=df.append(df2,ignore_index=True)
            else:
                df=df2
            df=df.sort_values(by=['mae'])
            df.to_csv(path_or_buf=fName, index=False)
            del df

In [9]:
def QiJCP(alpha):
    W0=np.genfromtxt("trainElementaryShapes/models_NInt200/QiJCP2019_0/curv_weights0.txt",delimiter=',')
    b0=np.genfromtxt("trainElementaryShapes/models_NInt200/QiJCP2019_0/curv_biases0.txt",delimiter=',')
    W1=np.genfromtxt("trainElementaryShapes/models_NInt200/QiJCP2019_0/curv_weights1.txt",delimiter=',')
    b1=np.genfromtxt("trainElementaryShapes/models_NInt200/QiJCP2019_0/curv_biases1.txt")
    
    ixoffset = [-1.73638881051374e-13,-1.51212375953946e-13,-1.73638881051374e-13,-1.73638881051374e-13,6.53670960006449e-08,-1.73638881051374e-13,-1.73638881051374e-13,-1.11910480882216e-13,-1.73638881051374e-13];
    igain = [1.99999999999931,1.9999999999994,1.99999999999931,1.99999999999931,2.00000026146842,1.99999999999931,1.99999999999931,1.99999999999955,1.99999999999931];
    iymin = -1;
    
    oxoffset = -0.222333388916681;
    oymin = -1;
    ogain = 4.49775;
    
    alpha1D=np.zeros(9)
    iD=0
    for k in range(0,3):
        for m in range(0,3):
            alpha1D[iD]=alpha[k,m]
            iD+=1
    
    alpha1D=igain*(alpha1D-ixoffset)+iymin
    
    L1=tanh(np.matmul(W0,alpha1D)+b0)
    K=np.matmul(L1.T,W1)+b1
    K=(K-oymin)/ogain+oxoffset;
    return K

In [ ]:
# import pickle
import pandas as pd
from pandas import read_csv
import os

def testCurvModels(alphaPath,NInt,modelPath="trainElementaryShapes/models_NInt",eps=1e-8,isPeriodic=False,modelType="_"):
    modelPath="%s%d"%(modelPath,NInt)
    alphaFile="trainElementaryShapes/testDatasets/%s.pkl"%alphaPath
    fName = open(alphaFile,'rb')
    data=pickle.load(fName)
    fName.close()
    NL=data["NL"]
    print(NL)

    wDir="trainElementaryShapes/testResults_NInt%d"%(NInt)
    if not os.path.exists(wDir):
        os.mkdir(wDir)
    fName_L1="%s/%s_eps%g_L1_Abs_Curv.dat"%(wDir,alphaPath,eps)
    fName_L2="%s/%s_eps%g_L2_Abs_Curv.dat"%(wDir,alphaPath,eps)
    fName_LMax="%s/%s_eps%g_LMax_Abs_Curv.dat"%(wDir,alphaPath,eps)
    fName_L1R="%s/%s_eps%g_L1_Rel_Curv.dat"%(wDir,alphaPath,eps)
    fName_L2R="%s/%s_eps%g_L2_Rel_Curv.dat"%(wDir,alphaPath,eps)
    fName_LMaxR="%s/%s_eps%g_LMax_Rel_Curv.dat"%(wDir,alphaPath,eps)

    models=os.listdir(modelPath)
    models = list(filter(lambda a: modelType in a, models))
    models.insert(0,"heightFunction33")
    models.insert(1,"heightFunction35")
    models.insert(2,"heightFunction")
    models.insert(3,"QiJCP")
    for model in models:
        if os.path.exists(fName_L1):
            df=pd.read_csv(fName_L1)
            df2=pd.read_csv(fName_L2)
            dfM=pd.read_csv(fName_LMax)
            dfR=pd.read_csv(fName_L1R)
            df2R=pd.read_csv(fName_L2R)
            dfMR=pd.read_csv(fName_LMaxR)
        else:
            df=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df.insert(n+1,"%d"%NL[n],[0.])
            df2=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df2.insert(n+1,"%d"%NL[n],[0.])
            dfM=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfM.insert(n+1,"%d"%NL[n],[0.])
            dfR=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfR.insert(n+1,"%d"%NL[n],[0.])
            df2R=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df2R.insert(n+1,"%d"%NL[n],[0.])
            dfMR=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfMR.insert(n+1,"%d"%NL[n],[0.])
                
        if df[df.iloc[:,0]==model].shape[0]<1:
            if model=="heightFunction33" or model=="heightFunction35" or model=="heightFunction37" or model=="heightFunction":
                hf=heightFunction()
            elif model[:3]=="sgd":
                mlpC=mlpCurvatureOld("%s/%s/model_Curv.h5"%(modelPath,model))
            elif model=="QiJCP":
                print("Evaluating MLP model of Qi et al. (2019).")
            else:
                try:
                    if model.find("Raw")>-1 and not model.find("symPrsv")>-1:
                        mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model),zoneModel=False)
                    elif model.find("normal")>-1:
                        mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model),symmetric=False,invertCurvature=False,positiveNormals=False)
                    else:
                        mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model))
                except:
                    continue
            print("new model: %s"%model)
            dfN=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfN.insert(n+1,"%d"%NL[n],[0.])
            df2N=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df2N.insert(n+1,"%d"%NL[n],[0.])
            dfMN=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfMN.insert(n+1,"%d"%NL[n],[0.])
            dfRN=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfRN.insert(n+1,"%d"%NL[n],[0.])
            df2RN=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df2RN.insert(n+1,"%d"%NL[n],[0.])
            dfMRN=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfMRN.insert(n+1,"%d"%NL[n],[0.])
                
            L1=np.zeros(len(NL))
            L2=np.zeros(len(NL))
            LMax=np.zeros(len(NL))
            L1R=np.zeros(len(NL))
            L2R=np.zeros(len(NL))
            LMaxR=np.zeros(len(NL))
            
            iC=0
            for N in data["NL"]:
                x2D=data["x2D_%d"%N]
                y2D=data["y2D_%d"%N]
                alpha=data["alpha_%d"%N]
                kappa=data["kappa_%d"%N]
                                        
                dx=x2D[1,0]-x2D[0,0]

                kappaML=np.zeros(x2D.shape)*np.nan
                
                Nx=x2D.shape[0]
                if isPeriodic:
                    iMax=0
                else:
                    iMax=3

                NInt=0.
                if model=="heightFunction33":
                    for i in range(iMax,np.mod(x2D.shape[0]-iMax,Nx)):
                        for j in range(1,y2D.shape[1]-1):
                            if eps<alpha[i,j]<1-eps:
                                NInt+=1.
                                young=reconScheme("young")
                                n=young.predict(alpha[i-1:i+2,j-1:j+2],dx)
                                nx=n[0];ny=n[1]
                                H=np.zeros(3)
                                if np.fabs(nx)<=np.fabs(ny): 
                                    for ii in range(-1,2,1):
                                        for jj in range(-1,2,1):
                                            H[ii+1]+=alpha[i+ii,j+jj]*dx
                                else:
                                    for jj in range(-1,2,1):
                                        for ii in range(-1,2,1):
                                            H[jj+1]+=alpha[i+ii,j+jj]*dx
                                Hx=(H[2]-H[0])/(2*dx)
                                Hxx=(H[2]-2.*H[1]+H[0])/dx**2
                                kappaML[i,j]=-Hxx/(1.+Hx**2)**1.5
                elif model=="heightFunction35":
                    for i in range(iMax,np.mod(x2D.shape[0]-iMax,Nx)):
                        for j in range(2,y2D.shape[1]-2):
                            if eps<alpha[i,j]<1-eps:
                                NInt+=1.
                                young=reconScheme("young")
                                n=young.predict(alpha[i-1:i+2,j-1:j+2],dx)
                                nx=n[0];ny=n[1]
                                if np.fabs(nx)<=np.fabs(ny):
                                    kappaML[i,j]=hf.predict(alpha[i-1:i+2,j-2:j+3],dx)
                                else:
                                    kappaML[i,j]=hf.predict(alpha[i-2:i+3,j-1:j+2],dx) 
                elif model=="heightFunction":
                    for i in range(iMax,np.mod(x2D.shape[0]-iMax,Nx)):
                        for j in range(3,y2D.shape[1]-3):
                            if eps<alpha[i,j]<1-eps:
                                NInt+=1.
                                young=reconScheme("young")
                                n=young.predict(alpha[i-1:i+2,j-1:j+2],dx)
                                nx=n[0];ny=n[1]
                                if np.fabs(nx)<=np.fabs(ny):
                                    kappaML[i,j]=hf.predict(alpha[i-1:i+2,j-3:j+4],dx)
                                else:
                                    kappaML[i,j]=hf.predict(alpha[i-3:i+4,j-1:j+2],dx) 
                elif model=="QiJCP":
                    for i in range(iMax,np.mod(x2D.shape[0]-iMax,Nx)):
                        for j in range(1,y2D.shape[1]-1):
                            if eps<alpha[i,j]<1-eps:
                                NInt+=1.
                                kappaML[i,j]=QiJCP(alpha[i-1:i+2,j-1:j+2])/dx
                                    
                else:
                    for i in range(iMax,np.mod(x2D.shape[0]-iMax,Nx)):
                        for j in range(2,y2D.shape[1]-2):
                            if eps<alpha[i,j]<1-eps:
                                NInt+=1.
                                kappaML[i,j]=mlpC.predict(alpha[i-2:i+3,j-2:j+3])/dx
                                
                L1[iC]=np.nansum(np.fabs(kappaML-kappa))/NInt 
                L2[iC]=np.sqrt(np.nansum((kappaML-kappa)**2.)/NInt)
                LMax[iC]=np.nanmax(np.fabs(kappaML-kappa))
                L1R[iC]=np.nansum(np.fabs((kappaML-kappa)/kappa))/NInt 
                L2R[iC]=np.sqrt(np.nansum((kappaML-kappa)**2./kappa**2.)/NInt)
                LMaxR[iC]=np.nanmax(np.fabs(kappaML-kappa))
                iC+=1

            dfN['name']=model
            dfN.iloc[0,1:]=L1
            if os.path.exists(fName_L1):
                df=df.append(dfN,ignore_index=True)
            else:
                df=dfN
            df=df.sort_values(by=['%s'%NL[-1]])
            df.to_csv(path_or_buf=fName_L1, index=False)
            del df
            
            df2N['name']=model
            df2N.iloc[0,1:]=L2
            if os.path.exists(fName_L2):
                df2=df2.append(df2N,ignore_index=True)
            else:
                df2=df2N
            df2=df2.sort_values(by=['%s'%NL[-1]])
            df2.to_csv(path_or_buf=fName_L2, index=False)
            del df2
            
            dfMN['name']=model
            dfMN.iloc[0,1:]=LMax
            if os.path.exists(fName_LMax):
                dfM=dfM.append(dfMN,ignore_index=True)
            else:
                dfM=dfMN
            dfM=dfM.sort_values(by=['%s'%NL[-1]])
            dfM.to_csv(path_or_buf=fName_LMax, index=False)
            del dfMN
            
            if 1:
                dfRN['name']=model
                dfRN.iloc[0,1:]=L1R
                if os.path.exists(fName_L1R):
                    dfR=dfR.append(dfRN,ignore_index=True)
                else:
                    dfR=dfRN
                dfR=dfR.sort_values(by=['%s'%NL[-1]])
                dfR.to_csv(path_or_buf=fName_L1R, index=False)
                del dfR
            
                df2RN['name']=model
                df2RN.iloc[0,1:]=L1R
                if os.path.exists(fName_L2R):
                    df2R=df2R.append(df2RN,ignore_index=True)
                else:
                    df2R=df2RN
                df2R=df2R.sort_values(by=['%s'%NL[-1]])
                df2R.to_csv(path_or_buf=fName_L2R, index=False)
                del df2R
            
                dfMRN['name']=model
                dfMRN.iloc[0,1:]=LMaxR
                if os.path.exists(fName_LMaxR):
                    dfMR=dfMR.append(dfMRN,ignore_index=True)
                else:
                    dfMR=dfMRN
                dfMR=dfMR.sort_values(by=['%s'%NL[-1]])
                dfMR.to_csv(path_or_buf=fName_LMaxR, index=False)
                del dfMR
    #return df


In [ ]:
import pickle
import pandas as pd
from pandas import read_csv
import os

def testCurvModelsSym(alphaPath,NInt,operation,modelPath="trainElementaryShapes/models_NInt",eps=1e-8,isPeriodic=False,modelType="_"):
    modelPath="%s%d"%(modelPath,NInt)
    alphaFile="trainElementaryShapes/testDatasets/%s.pkl"%alphaPath
    fName = open(alphaFile,'rb')
    data=pickle.load(fName)
    fName.close()
    NL=data["NL"]
    print(NL)

    wDir="trainElementaryShapes/testResults_NInt%d"%(NInt)
    if not os.path.exists(wDir):
        os.mkdir(wDir)
    fName_L1="%s/%s_eps%g_L1_Abs_Curv_%s.dat"%(wDir,alphaPath,eps,operation)
    fName_L2="%s/%s_eps%g_L2_Abs_Curv_%s.dat"%(wDir,alphaPath,eps,operation)
    fName_LMax="%s/%s_eps%g_LMax_Abs_Curv_%s.dat"%(wDir,alphaPath,eps,operation)

    models=os.listdir(modelPath)
    models = list(filter(lambda a: modelType in a, models))
    models.insert(0,"heightFunction")
    models.insert(1,"QiJCP")
    for model in models:
        if os.path.exists(fName_L1):
            df=pd.read_csv(fName_L1)
            df2=pd.read_csv(fName_L2)
            dfM=pd.read_csv(fName_LMax)
        else:
            df=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df.insert(n+1,"%d"%NL[n],[0.])
            df2=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df2.insert(n+1,"%d"%NL[n],[0.])
            dfM=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfM.insert(n+1,"%d"%NL[n],[0.])
                
        if df[df.iloc[:,0]==model].shape[0]<1:
            if model=="heightFunction33" or model=="heightFunction35" or model=="heightFunction37" or model=="heightFunction":
                hf=heightFunction()
            elif model[:3]=="sgd":
                mlpC=mlpCurvatureOld("%s/%s/model_Curv.h5"%(modelPath,model))
            elif model=="QiJCP":
                print("Evaluating MLP model of Qi et al. (2019).")
            else:
                try:
                    if model.find("Raw")>-1:
                        mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model),zoneModel=False)
                    elif model.find("normal")>-1:
                        mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model),symmetric=False,invertCurvature=False,positiveNormals=False)
                    elif model.find("odd")>-1:
                        mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model),symmetric=False,invertCurvature=False,positiveNormals=False,averageZones=False)
                    else:
                        mlpC=mlpCurvature("%s/%s/model_Curv.h5"%(modelPath,model))
                except:
                    continue
            print("new model: %s"%model)
            dfN=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfN.insert(n+1,"%d"%NL[n],[0.])
            df2N=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                df2N.insert(n+1,"%d"%NL[n],[0.])
            dfMN=pd.DataFrame({'name': ['model']})
            for n in range(len(NL)):
                dfMN.insert(n+1,"%d"%NL[n],[0.])

                
            L1=np.zeros(len(NL))
            L2=np.zeros(len(NL))
            LMax=np.zeros(len(NL))

            iC=0
            for N in data["NL"]:
                x2D=data["x2D_%d"%N]
                y2D=data["y2D_%d"%N]
                alpha=data["alpha_%d"%N]
                kappa=data["kappa_%d"%N]
                                        
                dx=x2D[1,0]-x2D[0,0]

                kappaML=np.zeros(x2D.shape)*np.nan
                kappaML2=np.zeros(x2D.shape)*np.nan
                
                Nx=x2D.shape[0]
                if isPeriodic:
                    iMax=0
                else:
                    iMax=3

                NInt=0.
                if model=="heightFunction":
                    for i in range(iMax,np.mod(x2D.shape[0]-iMax,Nx)):
                        for j in range(3,y2D.shape[1]-3):
                            if eps<alpha[i,j]<1-eps:
                                NInt+=1.
                                young=reconScheme("young")
                                n=young.predict(alpha[i-1:i+2,j-1:j+2],dx)
                                nx=n[0];ny=n[1]
                                if np.fabs(nx)<=np.fabs(ny):
                                    kappaML[i,j]=hf.predict(alpha[i-1:i+2,j-3:j+4],dx)
                                    if operation=="swap":
                                        kappaML2[i,j]=-hf.predict(1.-alpha[i-1:i+2,j-3:j+4],dx)
                                    elif operation=="refX":
                                        kappaML2[i,j]=hf.predict(np.fliplr(alpha[i-1:i+2,j-3:j+4]),dx)
                                    elif operation=="refY":
                                        kappaML2[i,j]=hf.predict(np.flipud(alpha[i-1:i+2,j-3:j+4]),dx)
                                    elif operation=="rot":
                                        kappaML2[i,j]=hf.predict(alpha[i-1:i+2,j-3:j+4].T,dx)
                                else:
                                    kappaML[i,j]=hf.predict(alpha[i-3:i+4,j-1:j+2],dx) 
                                    if operation=="swap":
                                        kappaML2[i,j]=-hf.predict(1.-alpha[i-3:i+4,j-1:j+2],dx)
                                    elif operation=="refX":
                                        kappaML2[i,j]=hf.predict(np.fliplr(alpha[i-3:i+4,j-1:j+2]),dx)
                                    elif operation=="refY":
                                        kappaML2[i,j]=hf.predict(np.flipud(alpha[i-3:i+4,j-1:j+2]),dx)  
                                    elif operation=="rot":
                                        kappaML2[i,j]=hf.predict(alpha[i-3:i+4,j-1:j+2].T,dx)  
                elif model=="QiJCP":
                    for i in range(iMax,np.mod(x2D.shape[0]-iMax,Nx)):
                        for j in range(1,y2D.shape[1]-1):
                            if eps<alpha[i,j]<1-eps:
                                NInt+=1.
                                kappaML[i,j]=QiJCP(alpha[i-1:i+2,j-1:j+2])/dx
                                if operation=="swap":
                                    kappaML2[i,j]=-QiJCP(1-alpha[i-1:i+2,j-1:j+2])/dx 
                                elif operation=="refX":
                                    kappaML2[i,j]=QiJCP(np.fliplr(alpha[i-1:i+2,j-1:j+2]))/dx 
                                elif operation=="refY":
                                    kappaML2[i,j]=QiJCP(np.flipud(alpha[i-1:i+2,j-1:j+2]))/dx   
                                elif operation=="rot":
                                    kappaML2[i,j]=QiJCP(alpha[i-1:i+2,j-1:j+2].T)/dx   
                else:
                    for i in range(iMax,np.mod(x2D.shape[0]-iMax,Nx)):
                        for j in range(2,y2D.shape[1]-2):
                            if eps<alpha[i,j]<1-eps:
                                NInt+=1.
                                kappaML[i,j]=mlpC.predict(alpha[i-2:i+3,j-2:j+3])/dx
                                if operation=="swap":
                                    kappaML2[i,j]=-mlpC.predict(1-alpha[i-2:i+3,j-2:j+3])/dx
                                elif operation=="refX":
                                    kappaML2[i,j]=mlpC.predict(np.fliplr(alpha[i-2:i+3,j-2:j+3]))/dx
                                elif operation=="refY":
                                    kappaML2[i,j]=mlpC.predict(np.flipud(alpha[i-2:i+3,j-2:j+3]))/dx 
                                elif operation=="rot":
                                    kappaML2[i,j]=mlpC.predict(alpha[i-2:i+3,j-2:j+3].T)/dx 
                                

                L1[iC]=np.nansum(np.fabs((kappaML-kappaML2)/kappa))/NInt 
                L2[iC]=np.sqrt(np.nansum((kappaML-kappaML2)**2./kappa**2.)/NInt)
                LMax[iC]=np.nanmax(np.fabs(kappaML-kappaML2))
                iC+=1

            dfN['name']=model
            dfN.iloc[0,1:]=L1
            if os.path.exists(fName_L1):
                df=df.append(dfN,ignore_index=True)
            else:
                df=dfN
            df=df.sort_values(by=['%s'%NL[-1]])
            df.to_csv(path_or_buf=fName_L1, index=False)
            del df
            
            df2N['name']=model
            df2N.iloc[0,1:]=L2
            if os.path.exists(fName_L2):
                df2=df2.append(df2N,ignore_index=True)
            else:
                df2=df2N
            df2=df2.sort_values(by=['%s'%NL[-1]])
            df2.to_csv(path_or_buf=fName_L2, index=False)
            del df2
            
            dfMN['name']=model
            dfMN.iloc[0,1:]=LMax
            if os.path.exists(fName_LMax):
                dfM=dfM.append(dfMN,ignore_index=True)
            else:
                dfM=dfMN
            dfM=dfM.sort_values(by=['%s'%NL[-1]])
            dfM.to_csv(path_or_buf=fName_LMax, index=False)
            del dfMN
    #return df